In [1]:
import __init__
from eval import eval, load_data, cv2_to_pil, pil_to_cv2
from dehaze import dehaze_and_enhance
from datasets import load_dataset
from collections import defaultdict
from tqdm import tqdm
from eval_with_ground import evaluate_image_pair_from_pil
from eval_without_ground import evaluate_image
import numpy as np
import cv2
from PIL import Image

In [2]:
paired_samples = load_data()

100%|██████████| 7200/7200 [01:30<00:00, 79.18it/s] 


In [8]:
def eval(dataset, method):
    samples = paired_samples[dataset]
    results = defaultdict(list)
    for sample in tqdm(samples):
        low_image = sample["low_image"]
        high_image = sample.get("high_image")
        # print(low_image, high_image)
        if low_image is not None:
            reference_img = cv2_to_pil(method(pil_to_cv2(low_image)))
            # reference_img.show()
            if high_image is not None:
                scores = evaluate_image_pair_from_pil(reference_img, high_image)
                # Compute metrics here
            else:
                scores = evaluate_image(reference_img, low_image)
            for key, s in scores.items():
                    results[key].append(s)
    return results

In [4]:
paired_samples.keys()

dict_keys(['lol_dataset', 'Dark_Face', 'LOL-v2'])

### dahaze

In [11]:
result = eval('lol_dataset', dehaze_and_enhance)

100%|██████████| 500/500 [00:14<00:00, 33.38it/s]


In [ ]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 3874.5530 ± 3863.0906
PSNR: 14.2080 ± 4.2667
SSIM: 0.5058 ± 0.1517


: 

In [8]:
result = eval('Dark_Face', dehaze_and_enhance)

100%|██████████| 6000/6000 [07:36<00:00, 13.14it/s]


In [9]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 0.9235 ± 6.6519
CEI: 1.8999 ± 0.4302


In [9]:
result = eval('LOL-v2', dehaze_and_enhance)

100%|██████████| 100/100 [00:02<00:00, 41.16it/s]


In [10]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 2110.1587 ± 1784.0054
PSNR: 16.2350 ± 3.3487
SSIM: 0.5454 ± 0.1242
